In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import os
import requests
import json
import asyncio
import aiohttp
import us

### The following is EPA data on energy usage/capacity from 2023 

In [84]:
excel_file = pd.ExcelFile('Data/egrid2023_data_rev2.xlsx') # epa data from https://www.epa.gov/egrid/detailed-data 
dfs = {sheet: excel_file.parse(sheet, skiprows=1) for sheet in excel_file.sheet_names}

In [85]:
region_production = dfs["NRL23"] # NERC regional data

### Regional Corporation breakdown

In [86]:
region_cap_gen = region_production[["NERCNAME", "NRNAMEPCAP", "NRNGENAN"]]

In [87]:
region_cap_gen = region_cap_gen.rename(columns={"NERCNAME": "region_name", 
                                                "NRNAMEPCAP": "inst_capacity (MW)", 
                                                "NRNGENAN": "yearly_gen (MWh)"})

In [88]:
region_cap_gen

,region_name,inst_capacity (MW),yearly_gen (MWh)
0,Alaska,3095.5,6.678905e+06
1,Hawaii,3451.2,9.158618e+06
2,Midwest Reliability Organization,165769.9,4.705173e+08
3,Northeast Power Coordinating Council,82817.8,2.261481e+08
4,Puerto Rico,6532.7,1.750115e+07
5,Reliability First Corporation,250781.4,9.091147e+08
6,SERC Reliability Corporation,385791.2,1.333721e+09
7,Texas Regional Entity,144275.6,4.721223e+08
8,Western Electricity Coordinating Council,261053.5,7.460087e+08


In [89]:
hours_per_year = 365 * 24
region_cap_gen["capacity_factor"] = region_cap_gen["yearly_gen (MWh)"] / (region_cap_gen["inst_capacity (MW)"] * hours_per_year)

In [90]:
region_cap_gen.sort_values(by="capacity_factor", ascending=False)

,region_name,inst_capacity (MW),yearly_gen (MWh),capacity_factor
5,Reliability First Corporation,250781.4,9.091147e+08,0.413827
6,SERC Reliability Corporation,385791.2,1.333721e+09,0.394647
7,Texas Regional Entity,144275.6,4.721223e+08,0.373558
8,Western Electricity Coordinating Council,261053.5,7.460087e+08,0.326220
2,Midwest Reliability Organization,165769.9,4.705173e+08,0.324016
3,Northeast Power Coordinating Council,82817.8,2.261481e+08,0.311720
4,Puerto Rico,6532.7,1.750115e+07,0.305823
1,Hawaii,3451.2,9.158618e+06,0.302939
0,Alaska,3095.5,6.678905e+06,0.246303


### State breakdown 

In [91]:
state_production = dfs["ST23"]

In [92]:
state_cap_gen = state_production[["PSTATABB", "STNAMEPCAP", "STNGENAN"]].copy()

In [93]:
state_cap_gen["capacity_factor"] = state_cap_gen["STNGENAN"] / (state_cap_gen["STNAMEPCAP"] * hours_per_year)

In [94]:
state_cap_gen.head(10)

,PSTATABB,STNAMEPCAP,STNGENAN,capacity_factor
0,AK,3095.5,6.678905e+06,0.246303
1,AL,32924.0,1.385963e+08,0.480546
2,AR,17060.1,6.319565e+07,0.422865
3,AZ,33107.6,1.118209e+08,0.385559
4,CA,96731.0,2.160802e+08,0.255003
5,CO,21370.0,5.747149e+07,0.307004
6,CT,10965.2,4.065207e+07,0.423216
7,DC,56.6,1.718695e+05,0.346640
8,DE,3441.0,4.772059e+06,0.158313
9,FL,76100.7,2.580990e+08,0.387163


### Below are the states with highest usage as a percentage of instantaneous capacity

In [134]:
top6states = state_cap_gen.sort_values(by="capacity_factor", ascending=False).head(6)
top6states

,PSTATABB,STNAMEPCAP,STNGENAN,capacity_factor
38,PA,53762.0,2.359126e+08,0.500924
25,MS,17031.6,7.293344e+07,0.488840
28,ND,9875.2,4.206881e+07,0.486307
40,RI,2472.6,1.041655e+07,0.480912
1,AL,32924.0,1.385963e+08,0.480546
35,OH,33503.5,1.331211e+08,0.453579


### We want to also look at the peak energy usage by each of these states compared with grid capacity rather than total - that might give us a better sense for who is least prepared off in the worst case scenarios. Below is slightly more granular data 

In [96]:
load_dotenv()
api_key = os.getenv('EIA_API_KEY')

### Exploring summer months for an example of what peak energy usage might look like in a given year.

In [126]:
base_url = 'https://api.eia.gov/v2/electricity/rto/region-sub-ba-data/data/'
all_data = []
offset = 0
parents = ["PJM", "MISO", "SPP", "ISONE", "TVA"]
start = "2023-06-01T00"
end = "2023-08-31T23"
length = 5000 

In [127]:
async def fetch_page(session, parent, offset):
    params = {
        "api_key": api_key,
        "frequency": "hourly",
        "data[0]": "value",
        "start": start,
        "end": end,
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": offset,
        "length": length
    }
    async with session.get(base_url, params=params) as resp:
        resp.raise_for_status()
        return await resp.json()

In [133]:
async def fetch_all_for_parent(session, parent):
    offset = 0
    parent_data = []
    while True:
        data = await fetch_page(session, parent, offset)
        rows = data.get("response", {}).get("data", [])
        if not rows:
            break
        parent_data.extend(rows)
        if len(rows) < length:
            break
        offset += length
    print(f"Fetched {len(parent_data)} rows for {parent}")
    return parent_data

async def main():
    all_data = []
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_all_for_parent(session, parent) for parent in parents]
        results = await asyncio.gather(*tasks)

        for res in results:
            all_data.extend(res)
            
    return all_data


In [129]:
all_data = await main()

Fetched 179170 rows for PJM
Fetched 179170 rows for ISONE
Fetched 179170 rows for SPP
Fetched 179170 rows for MISO
Fetched 179170 rows for TVA


In [142]:
eia_energy_data = pd.DataFrame(all_data)
eia_energy_data.to_csv("hourly_energy_data.csv")

In [178]:
# Map EIA parents to the corresponding reliability council
parent_to_region = {
    "PJM": "Reliability First Corporation",
    "ISONE": "Northeast Power Coordinating Council",
    "SPP": "Texas Regional Entity",  # approximate
    "MISO": "Midwest Reliability Organization",
    "TVA": "SERC Reliability Corporation"  # approximate
}

parent_capacity = {
    parent: region_cap_gen.loc[region_cap_gen["region_name"] == region, "inst_capacity (MW)"].values[0]
    for parent, region in parent_to_region.items()
}

In [182]:
parent_capacity_df = pd.DataFrame(list(parent_capacity.items()), columns=["parent", "inst_capacity_MW"])

In [184]:
eia_energy_data_cap = eia_energy_data.merge(parent_capacity_df, on="parent", how="left")

In [185]:
eia_energy_data_cap = eia_energy_data_cap[eia_energy_data_cap["parent"].isin(["PJM", "MISO", "ISONE", "TVA"])].copy()

In [186]:
eia_energy_data_cap["capacity_factor"] = eia_energy_data_cap["value"].astype(float) / eia_energy_data_cap["inst_capacity_MW"]

In [187]:
eia_energy_data_cap.sort_values(by="capacity_factor", ascending=False).head(10)

,Unnamed: 0,period,subba,subba-name,parent,parent-name,value,value-units,inst_capacity_MW,capacity_factor
372305,372305,2023-08-23T22,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33917,megawatthours,165769.9,0.204603
13965,13965,2023-08-23T22,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33917,megawatthours,165769.9,0.204603
551475,551475,2023-08-23T22,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33917,megawatthours,165769.9,0.204603
730645,730645,2023-08-23T22,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33917,megawatthours,165769.9,0.204603
193135,193135,2023-08-23T22,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33917,megawatthours,165769.9,0.204603
730563,730563,2023-08-23T23,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33870,megawatthours,165769.9,0.204319
551393,551393,2023-08-23T23,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33870,megawatthours,165769.9,0.204319
13883,13883,2023-08-23T23,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33870,megawatthours,165769.9,0.204319
372223,372223,2023-08-23T23,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33870,megawatthours,165769.9,0.204319
193053,193053,2023-08-23T23,8910,"Zones 8, 9 and 10 - MISO",MISO,"Midcontinent Independent System Operator, Inc.",33870,megawatthours,165769.9,0.204319


### The summer months we looked at don't show the highest peak hourly usage, since we had average usages that were higher earlier. Those can still be a reasonable gauge though, since average usage being close to 50% in Pennsylvania gives us a sense that peak usage must be higher than that at least.

### Waste Breakdowns

In [136]:
waste_excel_file = pd.ExcelFile('Data/Compiled State Public Waste Data.xlsx') # from https://catalog.data.gov/dataset/publicly-reported-state-municipal-solid-waste-data-version-1/resource/2d6a3af8-a773-4f63-b6b3-dcd6ca451ad5

In [140]:
waste_dfs = {sheet: waste_excel_file.parse(sheet, skiprows=1) for sheet in waste_excel_file.sheet_names}

In [197]:
waste_aggregates = waste_dfs["Raw Data Table"]
waste_aggregates = (waste_aggregates
                    [["State/Territory", "State % of US GDP", "Data year", "Total MSW Generated (tons)"]]
                    .dropna())
waste_aggregates = waste_aggregates[waste_aggregates["Total MSW Generated (tons)"] != "not found"].copy()
waste_aggregates["Total MSW Generated (tons)"] = waste_aggregates["Total MSW Generated (tons)"].astype(float)

In [200]:
state_to_code = {state.name: state.abbr for state in us.states.STATES}
waste_aggregates["PSTATABB"] = waste_aggregates["State/Territory"].map(state_to_code)

In [233]:
waste_plus_energy = (waste_aggregates
                     .merge(state_cap_gen, on="PSTATABB", how="left")
                    [["PSTATABB", "Data year", "Total MSW Generated (tons)", "STNAMEPCAP", "STNGENAN"]]
                    .dropna())

In [234]:
waste_plus_energy = waste_plus_energy.rename(columns={"Total MSW Generated (tons)":"muni_waste_tons",
                                                      "STNAMEPCAP": "inst_cap_MW",
                                                      "STNGENAN": "yearly_gen_MWh",
                                                      "PSTATABB": "state_code",
                                                      "Data year": "year"})

In [235]:
percent_processed = 0.85 # percent of waste that is actually processed on average
max_epa_guideline = 0.85 # Epa suggests max usage of grid should not exceed 85% 
energy_per_ton = 500e-3 # (500 kW)
waste_plus_energy["energy_created"] = waste_plus_energy["muni_waste_tons"] * 2 * percent_processed * energy_per_ton # 2x waste
waste_plus_energy["yearly_cap_MW"] = waste_plus_energy["inst_cap_MW"] * hours_per_year
waste_plus_energy["excess_energy_demand"] = (waste_plus_energy["yearly_gen_MWh"] * 2
                                             - max_epa_guideline * waste_plus_energy["yearly_cap_MW"])
waste_plus_energy["percent_of_excess"] = waste_plus_energy["energy_created"] / waste_plus_energy["excess_energy_demand"]

In [237]:
waste_plus_energy.sort_values(by="percent_of_excess", ascending=False).head(5)

,state_code,year,muni_waste_tons,inst_cap_MW,yearly_gen_MWh,energy_created,yearly_cap_MW,excess_energy_demand,percent_of_excess
22,SC,FY 2018,4289591.00,26563.6,1.008534e+08,3.646152e+06,232697136.0,3.914209e+06,0.931517
18,OH,2017,14107163.00,33503.5,1.331211e+08,1.199109e+07,293490660.0,1.677512e+07,0.714814
12,MT,2016,1803435.00,6890.3,2.689576e+07,1.532920e+06,60359028.0,2.486342e+06,0.616536
20,PA,2018,20652939.36,53762.0,2.359126e+08,1.755500e+07,470955120.0,7.151325e+07,0.245479
21,RI,2018,514425.00,2472.6,1.041655e+07,4.372612e+05,21659976.0,2.422126e+06,0.180528


### Above identifies locations where doubling demand for energy might exceed current grid capacity, and how much of that excess energy demand (over EPA guideliens) waste to energy conversions could alleviate